In [1]:
import os
import datetime
import random
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, Band
from bokeh.embed import components
from bokeh.models import Range1d

import numpy as np

In [2]:
import pandas as pd


In [3]:
def get_data():
    df = pd.read_csv('data/states.csv')
    df = df[(df['state'] == 'Washington')]
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by = ['date'])
    return df


In [10]:
def get_pop_data():
    df = pd.read_csv('data/seven_day_state.csv')
    #df = df[(df['state'] != 'Guam') & (df['state'] != 'Northern Mariana Islands')]
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by = ['date'])
    return df

def get_pop():
    df = get_pop_data()
    states = df['state']
    population = df['population']
    d = {}
    for x in set(list(zip(states, population))):
        d[x[0]] = x[1]
    return d

def get_data_sweden():
    df = pd.read_csv('data/sweden_vs.csv')
    df['date'] = pd.to_datetime(df['date'])
    df = df[df['region'] == 'Sweden']
    df = df.sort_values(by = 'date')
    return df


def get_data_germany():
    df = pd.read_csv('data/world.csv')
    df['date'] = pd.to_datetime(df['date'])
    df = df[df['country'] == 'Germany']
    df = df.sort_values(by = 'date')
    return df


def combine_wa_sweden():
    df_wa = get_data()
    us_pop = get_pop()
    df_wa = df_wa.assign(pop=us_pop['Washington'])
    df_wa = df_wa.assign(cases_100k = df_wa['cases']/df_wa['pop'] * 1e5)
    df_sweden = get_data_sweden()
    df_world = pd.read_csv('data/world.csv')
    df_world['date'] = pd.to_datetime(df_world['date'])
    df_world = df_world.assign(cases_100k = df_world['cases']/df_world['population'] * 1e5)
    return df_wa, df_world

get_data_sweden()
df_wa, df_world= combine_wa_sweden()


In [18]:
output_notebook()
def make_graph_over_time(window = 7):
    df_wa, df_world = combine_wa_sweden()
    dates_wa = df_wa['date']
    df_sweden = df_world[df_world['country'] == 'Sweden']
    df_netherlands = df_world[df_world['country'] == "Netherlands"]
    df_belgium = df_world[df_world['country'] == "Belgium"]

    dates_netherlands = df_netherlands['date']
    dates_belgium = df_belgium['date']

    cases_netherlands = df_netherlands['cases_100k'].rolling(window).mean()
    cases_belgium = df_belgium['cases_100k'].rolling(window).mean()

    dates_sweden = df_sweden['date']
    p = figure(x_axis_type = 'datetime')
    cases_wa = df_wa['cases_100k'].rolling(window).mean()
    cases_sweden = df_sweden['cases_100k'].rolling(window).mean()
    p.line(x = dates_wa, y = cases_wa, color = 'blue', legend_label = 'WA' )
    p.line(x = dates_sweden, y = cases_sweden, color = 'red', legend_label = 'Sweden')
    p.line(x = dates_netherlands, y = cases_netherlands, color = 'green', legend_label = "Netherlands")
    #p.line(x = dates_belgium, y = cases_belgium, color = 'black', legend_label = "Belgium")

    p.legend.location = 'top_left'
    p.yaxis.axis_label = 'Cases/100K'

    return p

show(make_graph_over_time())

Loading BokehJS ...